In [11]:
from tqdm import tqdm

from datetime import date, timedelta
import pandas as pd
import numpy as np
from src.financial_crawler.fetch_daily_exchange import get_daily_exchange_info, get_stock_list
from src.financial_crawler.fetch_financial_report import fetch_monthly_revenue, fetch_seasonal_report
from src.financial_crawler.process_monthly_revenue import process_monthly_revenue
from src.financial_crawler.process_comprehensive_income import process_comprehensive_income
from src.financial_crawler.process_balance_sheet import process_balance_sheet
from src.financial_crawler.process_cash_flow import process_cash_flow

from src.news_crawler.get_anue_daily_news import get_anue_news
from src.news_crawler.get_yahoo_daily_news import get_yahoo_news

KeyError: 'DB_CONNECTION'

In [15]:
pd.DataFrame({'x': [1, 2, 3]})['x'].iloc[0]

1

In [2]:
with open('./data/stock_list.txt', 'r', encoding='UTF-8') as f:
    stock_list = [line.rstrip('\n') for line in f]

In [3]:
TODAY = date.today()
today_str = TODAY.strftime('%Y%m%d')
# today_str = '20250117'

YEAR = TODAY.year
MONTH = TODAY.month
DAY = TODAY.day

In [12]:
df_stocks = get_stock_list(today_str)

========== 取得股票清單 ==========


In [9]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

host = "35.233.216.241"
port = 3306
username = "barrywu"
password = "Barrywoo576%&^"
database = 'tw_stock_agent'
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

In [10]:
engine.connect()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '35.199.149.222' (timed out)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

## 每日成交資訊

In [5]:
df_daily_exchange_info = get_daily_exchange_info(today_str)

========== 取得股票清單 ==========
========== 成交資訊 ==========
========== 三大法人買賣超 ==========


In [7]:
df_daily_exchange_info.head(2)

,stat_date,證券代號,證券名稱,類型,上市櫃,產業,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,外陸資買賣超股數,投信買賣超股數,自營商買賣超股數,三大法人買賣超股數
0,2025-01-17,1101,台泥,股票,上市,水泥工業,15983898.0,6070.0,503018876.0,31.1,31.85,31.05,31.7,0.75,26.64,1470754.0,312000.0,183923.0,1966677.0
1,2025-01-17,1102,亞泥,股票,上市,水泥工業,5042990.0,2258.0,208376439.0,40.9,41.60,40.90,41.4,0.50,14.08,-538776.0,39767.0,1513991.0,1014982.0


In [ ]:
df_daily_exchange_info.to_sql('daily_exchange_info', con=engine, if_exists='append', index=False)

```
CREATE TABLE daily_exchange_info (
    stat_date DATE NOT NULL,
    證券代號 VARCHAR(16) NOT NULL,
    證券名稱 VARCHAR(16) NOT NULL,
    類型 VARCHAR(4) NOT NULL,
    上市櫃 VARCHAR(4) NOT NULL,
    產業 VARCHAR(10) NOT NULL,
    成交股數 FLOAT DEFAULT NULL,
    成交筆數 FLOAT DEFAULT NULL,
    成交金額 FLOAT DEFAULT NULL,
    開盤價 FLOAT DEFAULT NULL,
    最高價 FLOAT DEFAULT NULL,
    最低價 FLOAT DEFAULT NULL,
    收盤價 FLOAT DEFAULT NULL,
    漲跌價差 FLOAT DEFAULT NULL,
    本益比 FLOAT DEFAULT NULL,
    外陸資買賣超股數 FLOAT DEFAULT NULL,
    投信買賣超股數 FLOAT DEFAULT NULL,
    自營商買賣超股數 FLOAT DEFAULT NULL,
    三大法人買賣超股數 FLOAT DEFAULT NULL,
    PRIMARY KEY ('stat_date','證券代號')
)
```

## 月營收

In [3]:
if MONTH == 1:  # 拿去年12月
    fetch_monthly_revenue(year=YEAR-1, month=12)
    df_monthly_rev = process_monthly_revenue(year=YEAR-1, month=12)
else:
    MONTH -= 1
    fetch_monthly_revenue(year=YEAR, month=MONTH-1)
    df_monthly_rev = process_monthly_revenue(year=YEAR, month=MONTH-1)

獲取2024年12月營收
(1761, 7)


In [14]:
df_monthly_rev.head(2)

,stat_date,證券代號,證券名稱,年度,月份,營收,備註
0,2024-12-01,1101,台泥,2024,12,16543379,2024/3/6起併入OYAK及Cimpor，以致營收較去年同期增加。
1,2024-12-01,1102,亞泥,2024,12,7085373,<NA>


In [18]:
df_monthly_rev.to_sql('monthly_revenue', con=engine, if_exists='append', index=False)

654825418

```
CREATE TABLE monthly_revenue (
    stat_date DATE NOT NULL,
    證券代號 VARCHAR(16) NOT NULL,
    證券名稱 VARCHAR(16) NOT NULL,
    年度 SMALLINT NOT NULL,
    月份 TINYINT NOT NULL,
    營收 INT DEFAULT NULL,
    備註 VARCHAR(255) DEFAULT NULL,
    PRIMARY KEY (`stat_date`,`證券代號`)
)
```

## 財務報表

In [4]:
report_type = '綜合損益表'
for market_type in ['上市', '上櫃']:
    for report_type in ['綜合損益表', '資產負債表', '現金流量表']:
        if MONTH <= 4:  # 拿去年第四季
            fetch_season = 4
            fetch_year = YEAR - 1
        else:
            fetch_season = MONTH // 3 - 1  # 拿上季報表
            fetch_year = YEAR
        fetch_season = 3
        fetch_year = YEAR - 1
        # fetch_seasonal_report(fetch_year, fetch_season, market_type, report_type)

### 綜合損益表

In [5]:
df_comprehensive_income = process_comprehensive_income(fetch_year, fetch_season)

上市 保險業
上市 銀行業
上櫃 一般業
平均（營業收入－營業成本－營業毛利： 95.66
上市 證券業
上市 異業
上市 金控業
上櫃 金融保險業
上市 一般業
平均（營業收入－營業成本－營業毛利： -59.42


In [6]:
df_comprehensive_income.head(2)

,stat_date,證券代號,證券名稱,年度,季度,營業收入,營業成本,營業毛利,營業費用,營業利益,業外收支,稅前淨利,所得稅,稅後淨利,EPS
0,2024-07-01,2816,旺旺保,2024,3,8116878,5399755,2717123,1729784.0,987339.0,19324.0,1006663,160470,846193,3.78
1,2024-07-01,2832,台產,2024,3,5607813,3313595,2294218,1138725.0,1155493.0,-3114.0,1152379,164566,987813,2.73


```
CREATE TABLE comprehensive_income (
    stat_date DATE NOT NULL,
    證券代號 VARCHAR(16) NOT NULL,
    證券名稱 VARCHAR(16) NOT NULL,
    年度 SMALLINT NOT NULL,
    季度 TINYINT NOT NULL,
    營業收入 FLOAT DEFAULT NULL,
    營業成本 FLOAT DEFAULT NULL,
    營業毛利 FLOAT DEFAULT NULL,
    營業費用 FLOAT DEFAULT NULL,
    營業利益 FLOAT DEFAULT NULL,
    業外收支 FLOAT DEFAULT NULL,
    稅前淨利 FLOAT DEFAULT NULL,
    所得稅 FLOAT DEFAULT NULL,
    稅後淨利 FLOAT DEFAULT NULL,
    EPS FLOAT DEFAULT NULL,
    PRIMARY KEY (`stat_date`,`證券代號`)
)
```

### 現金流量表

In [8]:
df_cash_flow = process_cash_flow(fetch_year, fetch_season)

In [20]:
df_cash_flow.sample(2)

,stat_date,證券代號,證券名稱,年度,季度,營業現金流,投資現金流,籌資現金流,匯率變動之影響,淨現金流
256,2024-07-01,2258,鴻華先進-創,2024,3,-595761.0,-201810.0,-57522.0,1163.0,-853930.0
261,2024-07-01,2308,台達電,2024,3,51190776.0,-27037744.0,3529093.0,5306774.0,32988899.0


```
CREATE TABLE cash_flow (
    stat_date DATE NOT NULL,
    證券代號 VARCHAR(16) NOT NULL,
    證券名稱 VARCHAR(16) NOT NULL,
    年度 SMALLINT NOT NULL,
    季度 TINYINT NOT NULL,
    營業現金流 FLOAT DEFAULT NULL,
    投資現金流 FLOAT DEFAULT NULL,
    籌資現金流 FLOAT DEFAULT NULL,
    匯率變動之影響 FLOAT DEFAULT NULL,
    淨現金流 FLOAT DEFAULT NULL,
    PRIMARY KEY (`stat_date`,`證券代號`)
)
```

### 資產負債表

In [22]:
df_balance_sheet = process_balance_sheet(fetch_year, fetch_season)

上市 異業
上市 證券業
上櫃 一般業
上市 保險業
上市 銀行業
上市 一般業
上市 金控業
上櫃 金融保險業


In [28]:
df_balance_sheet.head(2)

,stat_date,證券代號,證券名稱,年度,季度,流動資產,非流動資產,資產總計,流動負債,非流動負債,負債總計,股本,資本公積,保留盈餘,權益總計,每股參考淨值
0,2024-07-01,1409,新纖,2024,3,145905758.0,69810599.0,215716357,136115383.0,30236322,166351705,16184093,2046829,14465270,49364652,23.71
1,2024-07-01,1718,中纖,2024,3,720407583.0,240218323.0,960625906,848254480.0,27982869,876237349,16859057,1712237,2881225,84388557,16.33


```
CREATE TABLE balance_sheet (
    stat_date DATE NOT NULL,
    證券代號 VARCHAR(16) NOT NULL,
    證券名稱 VARCHAR(16) NOT NULL,
    年度 SMALLINT NOT NULL,
    季度 TINYINT NOT NULL,
    流動資產 FLOAT DEFAULT NULL,
    非流動資產 FLOAT DEFAULT NULL,
    資產總計 FLOAT DEFAULT NULL,
    流動負債 FLOAT DEFAULT NULL,
    非流動負債 FLOAT DEFAULT NULL,
    負債總計 FLOAT DEFAULT NULL,
    股本 FLOAT DEFAULT NULL,
    資本公積 FLOAT DEFAULT NULL,
    保留盈餘 FLOAT DEFAULT NULL,
    權益總計 FLOAT DEFAULT NULL,
    每股參考淨值 FLOAT DEFAULT NULL,
    PRIMARY KEY (`stat_date`,`證券代號`)
)
```

## 新聞爬蟲

```
CREATE TABLE daily_news_crawl (
    id INT AUTO_INCREMENT PRIMARY KEY,
    stat_date DATE NOT NULL,
    標題 VARCHAR(255) DEFAULT NULL,
    時間 DATETIME DEFAULT NULL,
    關鍵字 VARCHAR(255) DEFAULT NULL,
    內容 TEXT DEFAULT NULL
)
```

### Anue

In [5]:
YESTERDAY = TODAY - timedelta(days=1)
yesterday_str = YESTERDAY.strftime('%Y-%m-%d')

In [6]:
df_anue_news = get_anue_news(yesterday_str)
df_anue_news.head(2)

,stat_date,標題,時間,關鍵字,內容
0,2025-01-28,〈蛇年產業景氣前瞻〉AI伺服器帶動被動元件需求激增 相關業者營運看旺,2025-01-28 03:40:03,"AI伺服器,AI PC,被動元件",AI 伺服器推升被動元件規格、用量以及單價大幅提升，包括國巨、華新科及旗下佳邦，還有三集瑞 ...
1,2025-01-28,藥華藥衛冕生技市值王 預計Ropeg今年陸續取得ET藥證,2025-01-28 02:40:06,"生技,藥華藥,Ropeg,ET,康霈*,保瑞,美時,順藥",藥華藥 (6446-TW) 龍年衛冕生技族群市值王，以龍年封關日收盤價 687 元計算，市值...


In [15]:
# df_anue_news.to_sql('anue_daily_news', con=engine, if_exists='append', index=False)

### Yahoo

In [46]:
df_stocks = get_stock_list(today_str)
stock_list = df_stocks.loc[(df_stocks['類型'] == '股票') & ~(df_stocks['產業'].isin(EXCLUDE_FIELDS)), '證券代號'].tolist()

========== 取得股票清單 ==========


In [36]:
EXCLUDE_FIELDS = [
    '水泥工業', '食品工業', '塑膠工業', '其他業', '紡織纖維', '運動休閒', '玻璃陶瓷',
    '居家生活', '橡膠工業', '電子通路業', '資訊服務業', '貿易百貨業', '數位雲端'
]

In [266]:
df_yahoo_news = []
for stock in tqdm(stock_list[:2]+['2330']):
    temp = get_yahoo_news(stock, yesterday_str)
    if isinstance(temp, pd.DataFrame):
        df_yahoo_news.append(temp)

100%|██████████| 3/3 [00:34<00:00, 11.45s/it]


In [270]:
pd.concat(df_yahoo_news)

,stat_date,標題,時間,關鍵字,內容
1,2025-01-28,DeepSeek衝擊台積電ADR 網友驚喜：過年發紅包還不快撿,2025-01-28 14:27:41+00:00,"鴻海,台積電",總部位於杭州的人工智慧公司「幻方量化」，旗下的「深度求索」（DeepSeek）公司過低成本硬...
2,2025-01-28,陸Deepseek屠殺AI股 陸行之曝對台積電、輝達衝擊,2025-01-28 03:08:44+00:00,"冠軍,台積電",DeepSeek最新AI模型引爆美國大型科技股暴跌，陸行之認為將結束近2年輝達吃肉，其他公司...
3,2025-01-28,台積電ADR遭血洗13％ 台股若開盤狂殺多少？謝金河曝驚悚答案,2025-01-28 02:32:40+00:00,"台積電,陽明,巨大",大陸AI創新公司「DeepSeek」（深度求索）標榜價廉物美，撼動華爾街，導致周一（27日）...
